In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import naive_bayes
from sklearn.metrics import confusion_matrix , roc_auc_score , classification_report,accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
sentim_data=pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
sentim_data.head(10)

In [ ]:
sentim_data.info()

# Cleaning the data

In [ ]:
sentim_data.dropna(inplace=True)

sentim_data.info()

In [ ]:
sns.countplot(x='sentiment', data=sentim_data)

In [ ]:
sentim_data.isnull().sum()

In [ ]:
vs=sentim_data['sentiment'].value_counts()
vs

In [ ]:
senti_nuteral=(sentim_data['sentiment']=='neutral').value_counts()
senti_pos=(sentim_data['sentiment']=='positive').value_counts()

senti_neg=(sentim_data['sentiment']=='negetive').value_counts()


data=list([senti_nuteral,senti_pos,senti_neg])
plt.violinplot(data)
senti_nuteral

In [ ]:
sentim_data['no._textword']=sentim_data['text'].apply(lambda x:len(str(x).split()))
sentim_data['no._sel_text']=sentim_data['selected_text'].apply(lambda x:len(str(x).split()))

In [ ]:
sentim_data.head()

In [ ]:
sentim_data['word_diff']=sentim_data['no._textword']-sentim_data['no._sel_text']

In [ ]:
sentim_data.head()

In [ ]:
y1=sentim_data['no._textword']
y2=sentim_data['no._sel_text']
x=sentim_data['word_diff']
fig=plt.figure(figsize=(30,15))
plt.scatter(y1,y2)
plt.scatter(y1,x)
plt.legend(['y1','y2',x],loc='best')
plt.show()

In [ ]:
temp = sentim_data.groupby('sentiment').count()['text']
temp

In [ ]:
labels=['neutral','positive','negative']
size=[temp.neutral,temp.positive,temp.negative]
fig=plt.figure(figsize=(10,5))
my_explode = (0, 0.1, 0)
ax1=plt.subplot()
ax1.pie(size,labels=labels,autopct='%1.1f%%',shadow=True,startangle=15,explode=my_explode)

In [ ]:
len_tweet = sentim_data['text'].apply(lambda x:len(str(x)))
len_selected = sentim_data['selected_text'].apply(lambda x:len(str(x)))
len_sentiment=sentim_data['sentiment'].apply(lambda x:len(str(x)))
fig=plt.figure(figsize=(30,15))
sns.distplot(len_tweet,norm_hist=True,color='orange')
sns.distplot(len_selected,norm_hist=True,color='yellow')


In [ ]:
pcal=pd.get_dummies(sentim_data['sentiment'])
pcal

In [ ]:
sentim_data=pd.concat([sentim_data,pcal],axis=1)

In [ ]:
sentim_data.head()

In [ ]:
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text)
    text=re.sub(r'#','',text)
    text=re.sub(r'RT[\s]+','',text)
    return text

sentim_data['selected_text']=sentim_data['selected_text'].apply(cleanTxt)
sentim_data
     

# Apply the subjectivity and polarity of the sentiment of selected_text

# Polarity

It simply means emotions expressed in a sentence.

Emotions are closely related to sentiments. The strength of a sentiment or opinion is typically linked to the intensity of certain emotions, e.g., joy and anger.

Opinions in sentiment analysis are mostly evaluations(although not always).

According to consumer behavior research, evaluations can be broadly categorized into two types:
1. Rational evaluations
2. Emotional evaluations.

Rational evaluation: Such evaluations are from rational reasoning, tangible

beliefs, and utilitarian attitudes. For example, the following sentences

Express rational evaluations: “The voice of this phone is clear,” “This car is worth the price,” and “I am happy with this car.”

Emotional evaluation: Such evaluations are from non-tangible and emotional responses to entities which go deep into people’s state of mind.

For example, the following sentences express emotional evaluations: “I love iPhone,” “I am so angry with their service people” and “This is the best car ever built.”

To make use of these two types of evaluations in practice, we can design 5 sentiment ratings, emotional negative (-2), rational negative (-1), neutral (0), rational positive (+1), and emotional positive (+2). In practice, neutral often means no opinion or sentiment expressed.


# subjectivty

Subjectivity

Subjective sentence expresses some personal feelings, views, or beliefs.

An example

subjective sentence is “I like iPhone.” Subjective expressions come in many forms, e.g., opinions, allegations, desires, beliefs, suspicions, and speculations.

A subjective sentence may not express any sentiment.
For example, “I think that he went home” and “I want a camera that can take good photos” are a subjective sentences, but does not express any sentiment


In [ ]:
def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getpolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
sentim_data['Subjectivity']=sentim_data['selected_text'].apply(getsubjectivity)
sentim_data['polarity']=sentim_data['selected_text'].apply(getpolarity)

In [ ]:
sentim_data.head(15)

In [ ]:
plt.figure(figsize=(10,5))
# for i in range(0,sentim_data.shape[0]):
plt.scatter(sentim_data['polarity'],sentim_data['Subjectivity'],marker='4',color='orange')
    
plt.title('sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.grid()
plt.show()
# here you see the majority of the sentiment is greater than 0

In [ ]:
sentim_data.hist(bins=50 , figsize=(20,15))


In [ ]:
corr_matrix=sentim_data.corr()

In [ ]:
corr_matrix['polarity'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes =["polarity" , "positive" , "Subjectivity" , "no._sel_text","neutral","negative"]
scatter_matrix(sentim_data[attributes], figsize=(18,12))

In [ ]:
sample_data=[['Good case for the money !','positive'],
            ['Do not waste your money.','negetive'],
            ['Good product. Really good.Love it!','positive']]
small_sample=pd.DataFrame(sample_data,columns=['selected_text','sentiment'])

In [ ]:
count_vec=CountVectorizer(binary=False, stop_words='english', ngram_range=(1,1),max_features=50000)
count_vec.fit(small_sample.selected_text)
small_tranformed=count_vec.transform(small_sample.selected_text)

In [ ]:
small_sample

In [ ]:
from pandas import DataFrame
print(DataFrame(small_tranformed.A,columns=count_vec.get_feature_names()).to_string())

In [ ]:
type(small_tranformed)

In [ ]:
small_tranformed.A

In [ ]:
tfidf=TfidfTransformer(use_idf=True)
tfidf.fit(small_tranformed)
small_tfidf=tfidf.transform(small_tranformed)

In [ ]:
print(DataFrame(small_tfidf.A,columns=count_vec.get_feature_names()).to_string())

In [ ]:
lr=LogisticRegression(penalty='l2',C=.8,random_state=21)

In [ ]:
text_classifier=Pipeline([
    ('vectorizer',CountVectorizer(binary=False,stop_words='english',ngram_range=(1,2))),
    ('tfidf',TfidfTransformer(use_idf=True,)),
    ('clf',lr),
])

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(sentim_data.selected_text, sentim_data.sentiment, test_size=0.35, random_state=4)

In [ ]:
text_classifier.fit(X_train,y_train)

In [ ]:
X_test[0:4]


In [ ]:
y_test[0:4]

In [ ]:
predicted_test = text_classifier.predict(X_test)
predicted_proba_test = text_classifier.predict_proba(X_test)



In [ ]:

from sklearn import metrics
# for training data
predicted_train = text_classifier.predict(X_train)

y_train = y_train.astype('category')
print(metrics.classification_report(y_train, predicted_train,
    labels=y_train.cat.categories.tolist()))

metrics.confusion_matrix(y_train, predicted_train)



In [ ]:
# for testing data

predicted_test = text_classifier.predict(X_test)

y_test = y_test.astype('category')
print(metrics.classification_report(y_test, predicted_test,
    labels=y_test.cat.categories.tolist()))

metrics.confusion_matrix(y_test, predicted_test)

In [ ]:
text_classifier.predict_proba(['Dogs love us !'])

it means this text shows negetive predicted value 2.5%,neutral predicted value 9.2%,positive predicted value 88.1% 

# Neural Network ANN

In [ ]:
corpus=sentim_data['selected_text'].values

In [ ]:
X=count_vec.fit_transform(corpus)

In [ ]:
X.shape

In [ ]:
tfidf=TfidfTransformer()

In [ ]:
X=tfidf.fit_transform(X)

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
y=sentim_data['sentiment'].values

In [ ]:
y=le.fit_transform(y)

In [ ]:
y.shape

In [ ]:
from keras import models
from keras.layers import Dense

In [ ]:
model=models.Sequential()
model.add(Dense(16,activation="relu",input_shape=(X.shape[1],)))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
X_val=X[:5000]
X_train=X[5000:]

y_val=y[:5000]
y_train=y[5000:]

In [ ]:
X_train.shape ,y_train.shape

In [ ]:
hist=model.fit(X,y,batch_size=128,epochs=6,validation_data=(X_val,y_val))

In [ ]:
result=hist.history

In [ ]:
plt.plot(result['val_accuracy'],label='val_accuracy')
plt.plot(result['accuracy'],label='Train_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(result['val_loss'],label='val_loss')
plt.plot(result['loss'],label='Train_loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_val,y_val)

# Test Time

In [ ]:
test=pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
test['selected_text']=test['text'].apply(cleanTxt)

In [ ]:
test.head()

In [ ]:
X_test=test['selected_text']

In [ ]:
X_test.shape

In [ ]:
X_test=count_vec.transform(X_test)

In [ ]:
X_test=tfidf.transform(X_test)

In [ ]:
X_test.shape

In [ ]:
print(X_test[0])

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
# y_pred[y_pred>0.5]=1

In [ ]:
# y_pred=y_pred.astype('int')

In [ ]:
y_pred.shape

In [ ]:
ids=test['textID']
ids.shape

In [ ]:
my_submission = pd.DataFrame({'textID': test.textID.values, 'selected_text': test.selected_text})
my_submission



In [ ]:
import csv
my_submission.to_csv('submission.csv', index=False)